<a href="https://colab.research.google.com/github/heckelei/PromotionskollegModule6800_2020/blob/master/6800_Day4-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 4: Code used during lecture and lab assignment


## Instructions

The notebook combines 'code used during lecture' with the corresponding lab assignment (see further down)
Please add answers/discussion/comments to the notebook as comments or text box. Do not create another file in addition.
When you are done with your assignment, save the notebook in drive and add your last name to the name of the file
Upload your final notebook to https://uni-bonn.sciebo.de/s/mTpqLLBN9Wu71Ku latest September 30th, 2020. The password for access is the same as before



## Code used during lecture



In [ ]:
import numpy as np
import pandas as pd
import scipy as sc
from sklearn.linear_model import LinearRegression, Lasso
from sklearn import linear_model

In [ ]:
# Set the numpy random seed
np.random.seed(100)

First generate simulated data for known DGP

In [ ]:
# code for creating simulated data
N = 10000
K = 3
sig = 0.3
mean = np.random.randint(0,40,K)
# TODO repalce with automatic option to allow for different K
cov = np.array([[30,0,0],[0,5,0],[0,0,15]])
mean

In [ ]:
# create characteristics/explantory variables
X = np.random.multivariate_normal(mean, cov,N)
X.shape

In [ ]:
# Check that cov is correct
np.cov(X,rowvar=False)

In [ ]:
# Add a constant to X
Xc = np.concatenate([np.ones([N,1]),X],axis=1)
# Get beta values
beta = pd.DataFrame(np.random.randint(0, 20, (Xc.shape[1],1)),index=['const']+[f"beta{i}" for i in range(1,Xc.shape[1])],columns=['beta_true'])
beta


In [ ]:
# create random white noise error
errStd = 50
err = np.random.normal(0, errStd, (N,1))

# Create dependent variable y=Xb+e
Y = np.matmul(Xc,beta.values)+err
Y

Run LASSO on simulated data

Compare to the true data generating process


In [ ]:
# Estimate the model with Lasso 
# Check how alpha affects the estimation
res = beta.transpose().copy()
lstBetaCols = list(res.columns)

lstAlpha = np.logspace(-5, 1, num = 10, base = 2)
# Loop over a range of alpha value
for alpha in lstAlpha:
  # Estimate Lasso
  modLasso = Lasso(normalize=True, fit_intercept=True, alpha=alpha)
  modLasso.fit(X, Y[:,0])

  # Add estimated coef to result dataframe
  # Note, I rounded alpha in the variable name just to make it more readable
  res.loc[f'beta_hat_alpha_{round(alpha,3)}','alpha'] = alpha
  res.loc[f'beta_hat_alpha_{round(alpha,3)}','const'] = modLasso.intercept_
  res.loc[f'beta_hat_alpha_{round(alpha,3)}',lstBetaCols[1:]] = modLasso.coef_.transpose()

res

In [ ]:
# Plot the estimated coeficients across different levels of alpha

import matplotlib.pyplot as plt
fig, ax = plt.subplots();
for strBeta in ['beta1','beta2','beta3']:
  ax.plot(res['alpha'],res[strBeta],label=strBeta)

ax.plot([10e5,0.031250],[0,0],color='black')
ax.set_xscale('log')
ax.set_xlabel('alpha (log scale)')
ax.invert_xaxis()
ax.set_xlim(1,0.031250)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.legend();

Now add a additional variable that is correlated to one of the existing variables but does not have an effect on the dependent variable 

In [ ]:
beta.values.squeeze()

In [ ]:
# Lasso Simulation example
# 1) Create a variable (X4) that is linearly dependent with X3
#    The strength of the relationship is governed by the parameter "paramX4"
#    A white noise error term is added. X4 = paramX4 * X3 + err
# 2) The strength of the relationship (paramX4) is varied in a loop 
# 3) In each step X4 is created and the Lasso is estimated. Alpha in the 
#    Lasso estimation is automatically determined using LassoCV (best alpha
#    selected by cross-validation). 
# 4) Results are saved given: 
#     - The correlation coef between X4 and X3
#     - The estimated coef from Lasso
#   => This allows to see the highest coerrelation coef for which Lasso can 
#     recover the true model


from sklearn.linear_model import LassoCV

res = pd.DataFrame(columns=['corrcoef','beta1','beta2','beta3','beta4'])
res.loc['true_model',['beta1','beta2','beta3']] = beta.iloc[1:].values.squeeze()
for errStdX4 in [0,1,2,3,4,5,7.5,10,15,20,25,30,45,50,55,100,200]:
  paramX4 = 2 # This parameter specifies the strength of the relationship between 
              # X3 and X4
  X4 = paramX4*X[:,[2]]+np.random.normal(0, errStdX4, (N,1))

  XPlus4 = np.concatenate([X,X4],axis=1)

  res.loc[f"errStdX4_{errStdX4}",'corrcoef'] = np.corrcoef(XPlus4[:,2],XPlus4[:,3])[0,1]
  # print(XPlus4[1,:])

  modLassoCV = LassoCV(cv=5,normalize=True,fit_intercept=True)
  modLassoCV.fit(XPlus4,  Y[:,0])
  coef = modLassoCV.coef_
  res.loc[f"errStdX4_{errStdX4}",'alpha'] = modLassoCV.alpha_
  res.loc[f"errStdX4_{errStdX4}",['beta1','beta2','beta3','beta4']] = coef
res

# Lab 4a 

The first part of today's lab will have you see if you can recover the true DGP using a LASSO

Task:

- Below you will find a DGP similar to the one above
- Try to determine the optimal alpha value in order to recover the "best" specification
- Check if this "best" specification is close to the true DGP

In [ ]:
# re-run the DGP code to get a different set of coefficients and data

In [ ]:
from sklearn.datasets import make_spd_matrix

In [ ]:
# Call the function to generate data

# Set the random seed such that everybody gets the same dataset
np.random.seed(11)
# Get beta values
K = 5
errStd = 3
beta = pd.DataFrame(np.random.randint(-5, 5, (K+1,1)),index=['const']+[f"beta{i}" for i in range(1,K+1)],columns=['beta_true'])
print(beta)

# Sklearn function to generate a random symmetric, positive-definite covariance matrix
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_spd_matrix.html
Xcov = make_spd_matrix(K)
# Generate random mean for X variables 
Xmean = np.random.randint(-20,20,K)

# create characteristics/explantory variables
X = np.random.multivariate_normal(Xmean, Xcov,N)

# Add a constant to X
Xc = np.concatenate([np.ones([N,1]),X],axis=1)

# create random white noise error
err = np.random.normal(0, errStd, (N,1))

# Create dependent variable y=Xb+e
Y = np.matmul(Xc,beta.values)+err


# errStdXnoiseA = 20
# errStdXnoiseB = 30

errStdXnoiseA = 2
errStdXnoiseB = 2
            
# XnoiseA = 2*X[:,[1]]+2*X[:,[2]]+np.random.normal(0, errStdXnoiseA, (N,1))
# XnoiseB = 2*X[:,[2]]+1*X[:,[4]]+np.random.normal(0, errStdXnoiseB, (N,1))
XnoiseA = X[:,[1]]*X[:,[3]]*np.random.normal(0, errStdXnoiseB, (N,1))+np.random.normal(0, 1, (N,1))
XnoiseB = X[:,[2]]*X[:,[4]]*np.random.normal(0, errStdXnoiseB, (N,1))+np.random.normal(0, 2, (N,1))

XNoise = np.concatenate([X,XnoiseA, XnoiseB],axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
#  This randomly splits the data into 80% train and 20% test data
X_train, X_test, Y_train, Y_test = train_test_split(XNoise, Y, test_size = 0.2)


In [ ]:
# ================================
#  Task
# ================================

# Test several alpha values and check how this effects the R2 in the test set

# Prepare dataframe to hold results (no need to change anything here)
lstBeta = [f"beta{k}" for k in range(1,K+1)]
res = pd.DataFrame(columns=lstBeta+['betaNoiseA','betaNoiseB'])
res.loc['true_model',lstBeta] = beta.iloc[1:].values.squeeze()

# This is the place to set alpha values that sould be tested
lstAlpha = [...]

for alpha in  lstAlpha:

  # Specify the model you want to use
  modLasso = ...
  # Fit the model to data
  modLasso. ...
  
  # Get the coeficients of the model
  coef = 

  # Save results in list (no need to change anything here)
  res.loc[f"alpha_{alpha}",lstBeta+['betaNoiseA','betaNoiseB']] = coef
  res.loc[f"alpha_{alpha}",'alpha'] = alpha
  res.loc[f"alpha_{alpha}",'R2_train'] = modLasso.score(X_train, Y_train[:,0])
  res.loc[f"alpha_{alpha}",'R2_test'] = modLasso.score(X_test, Y_test[:,0])

res


In [ ]:
# ================================
#  Optional: Alternative approach using LassoCV 
# ================================
# One alternative to setting the alphas manually is to use 
# CV this is implemented in LassoCV


# TODO Try to implement a LassoCV estimation


# Prepare a dataframe to hold results 
# (no need to change something here, as long as you call your model "model")
print('Best Alpha: ', model.alpha_)
lstBeta = [f"beta{k}" for k in range(1,K+1)]
res = pd.DataFrame(columns=lstBeta+['betaNoiseA', 'betaNoiseB'])
res.loc['true_model',lstBeta] = beta.iloc[1:].values.squeeze()
res.loc['beta_hat',:] = model.coef_
res
# You can also check the example here:
#  https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html
# And try to prepare similar plots

# Lab 4b

The objective of the second part of today's lab is to determine the effects of protected areas on forest cover. To do this, we take the following strategy:

1) We estimate a model predicting forest cover where we use all of the observations that are not in a protected area. 

2) We use this model to predict forest cover for *all* observations. This gives as a prediction about the expected forest cover given the characteristics of the cell (such as slope, elevation, etc, just not protected status).

3) We compare the predicted forest cover with the actual forest cover. The difference between the two is the estimated effect of forest protected areas. 

=> Most of the steps for the estimation should be familiar from previous labs.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

In [ ]:
# Download data for amazon AWS store (this should only take 10s independent 
# of your internet connection)
!wget https://ilrmlcourse2020.s3.eu-central-1.amazonaws.com/brazil_all_data_v2.gzip

# load data into dataframe
df = pd.read_parquet('brazil_all_data_v2.gzip')

In [ ]:
# Define target (dependent) variable (% forest cover for 2018)
strY = 'perc_treecover'

# Define a list of features names (explantory variables)
lstX = [
  # 'wdpa_2017', => Exclude protected areas 
  'population_2015',
  'chirps_2017',
  'maize',
  'soy',
  'sugarcane',
  'perm_water',
  'travel_min',
  'cropland',
  'mean_elev',
  'sd_elev',
  'near_road',
  # 'wdpa_2017_lag_1st_order',
  'chirps_2017_lag_1st_order',
  'population_2015_lag_1st_order',
  'maize_lag_1st_order',
  'soy_lag_1st_order',
  'sugarcane_lag_1st_order',
  'perc_treecover_lag_1st_order',
  'perm_water_lag_1st_order',
  'travel_min_lag_1st_order',
  'cropland_lag_1st_order',
  'mean_elev_lag_1st_order',
  'sd_elev_lag_1st_order',
  'near_road_lag_1st_order',
 ]

In [ ]:
# Split that sample in those observations that ...
#... are protected areas
df_PA = df.loc[~(df['wdpa_2017']==0),:]

#... are NOT protected areas
df_NoPA = df.loc[df['wdpa_2017']==0,:]

print('Number of observations without protected area', df_NoPA.shape[0])
print('Number of observations with protected area', df_PA.shape[0])

In [ ]:
# Select the target variable 
# ... for the observations without protected areas 
Y_all_NoPA = df_NoPA[strY]
# ... for the observations with protected areas 
Y_PA = df_PA[strY]

# Get the features
# ... for the observations without protected areas 
X_all_NoPA =  df_NoPA.loc[:,lstX]
# ... for the observations wit protected areas 
X_PA_raw =  df_PA.loc[:,lstX]

Choose a ML model and features to predict deforestation

In [ ]:
# Split the data into train and test using only the the observations 
# with protected areas
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(X_all_NoPA, Y_all_NoPA, test_size = 0.2)

In [ ]:

# Scale data to 0-1 range using sklearn MinMaxScalar object 
# (see: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) 
scaler = MinMaxScaler()
# Use only the train data to fit the MinMaxScalar 
scaler.fit(X_train_raw)

# Apply the MinMax transformation to the train and test data 
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)


In [ ]:
# Define a function to print model stats
def printModStats(mod,X_train, Y_train,X_test, Y_test, showPlot=True):
  # Inspect the model performancefor those observations that are
  # not protected areas
  print('Score in train', mod.score(X_train, Y_train))
  print('Score in test', mod.score(X_test, Y_test))

  # Get predicted values for the test set
  Y_test_had_Tree = mod.predict(X_test)

  # Calculate MSE in test set
  mse_ols_sklearn  = mean_squared_error(Y_test,Y_test_had_Tree)
  print('\nMean squared error: ',mse_ols_sklearn)
  # The coefficient of determination: 1 is perfect prediction
  R2_ols_sklearn = r2_score(Y_test,Y_test_had_Tree)
  print('Coefficient of determination: ',R2_ols_sklearn)

  if showPlot:
    # plot Y vs Y-hat
    h = sns.jointplot(Y_test_had_Tree, Y_test, kind="hex")
    h.set_axis_labels('Y predicted', 'Y true');



In [ ]:
# Run XGBoost. In contrast to day 2 we now use XGBRegressor as our task is 
# a regression task and not a classification task
import xgboost as xgb
model_xgb = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
model_xgb.fit(X_train, Y_train)


In [ ]:
printModStats(model_xgb,X_train, Y_train,X_test, Y_test, showPlot=True)

Now we use the trained model to make predictions for those areas without protected areas

In [ ]:
# First we need to scale the data for those observations without 
# protected areas
X_PA = scaler.transform(X_PA_raw)

In [ ]:
# Lets define a function for our conterfactual that can be reused below 
def calculateConterfactual(mod,X_PA):
  # Then use the train model to make predictions for protected areas
  Y_PA_had = mod.predict(X_PA)

  # Now we can compare the mean forest cover in protected area that we observer
  print(f'Predicted mean forest cover in protected Areas {np.mean(Y_PA_had):0.2f} %' )
  print(f'Actual mean forest cover in protected Areas {np.mean(Y_PA):0.2f} %', )
  print(f'Mean difference between predicted and observed forest cover: {np.mean(Y_PA-Y_PA_had):0.2f} pp')


In [ ]:
# Call function for our xgb model 
calculateConterfactual(model_xgb,X_PA)

In [ ]:
# ================================
#  Task
# ================================

# Until now we have always used the default hyperparameters for XGBoost
# Lets see if we can improve model performance by optimizing some of those.
# Usually this is a rather complex and time consuming task with many 
# different ways to approach this. Here we only try to optimize two paramters
# manually. Specifically we want to optimize "max_depth" (Typical values: 3-10) 
# and "min_child_weight" (Typical values: 1-5).

# Hint: The caculation is quite time consuming. Coordinate in your team 
#       to try as many combination as possible.

# In case you are interested you can find a complete guide for doing 
# hyperparameter optimization for XGB here: 
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
# This is NOT required for this task here, just to give you an idea of how
# many paramter can be optimized (Note, this referes to an older sklearn version, 
# therefore some of the code might need to be adjusted) 

# Specify model with "max_depth" and "min_child_weight" set explicitly
model_xgb_opt = ...

# Fit model to train data 
model_xgb_opt....

# Print stats
printModStats(model_xgb_opt,X_train, Y_train,X_test, Y_test, showPlot=True)

# Do conterfactual 
calculateConterfactual(model_xgb_opt,X_PA)

In [ ]:
# ================================
#  Optional Task
# ================================

# Try alternative ML models and check how they perform compared to XGBoost. 
# If you are using sklearn you often only have to replace one line in oder to 
# change your model. LightGBM for example is an alterantive to XGBoost that 
# gained popularity.  
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor.  

## Alternative approach 

- Train a model with "protected areas" included 
- Select only the protected areas (in the test set)
- make a prediction for those areas for forest cover
- make a prediction for those areas for forest cover where we set the explanatory variable for "protected areas" equal to zero i.e. we make a prediction for forest cover under the assumption that those areas where not protected

In [ ]:
# Define binary variable for deforestration in 2018

Y_all = df['perc_treecover']
# Define a list of features names (explantory variables)
lstX = [
   'wdpa_2017', 
  'population_2015',
  'chirps_2017',
  'maize',
  'soy',
  'sugarcane',
  'perm_water',
  'travel_min',
  'cropland',
  'mean_elev',
  'sd_elev',
  'near_road',
  'wdpa_2017_lag_1st_order',
  'chirps_2017_lag_1st_order',
  'population_2015_lag_1st_order',
  'maize_lag_1st_order',
  'soy_lag_1st_order',
  'sugarcane_lag_1st_order',
  'perc_treecover_lag_1st_order',
  'perm_water_lag_1st_order',
  'travel_min_lag_1st_order',
  'cropland_lag_1st_order',
  'mean_elev_lag_1st_order',
  'sd_elev_lag_1st_order',
  'near_road_lag_1st_order',
 ]

# Get the explanatory Variables
X_all =  df.loc[:,lstX]

In [ ]:


# Split the data into train and test using only the the observations 
# with protected areas
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(X_all, Y_all, test_size = 0.2)
# Scale data to 0-1 range using sklearn MinMaxScalar object 
scaler = MinMaxScaler()
# Use only the train data to fit the MinMaxScalar 
scaler.fit(X_train_raw)

# Apply the MinMax transformation to the train and test data 
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [ ]:

model_xgb = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
model_xgb.fit(X_train, Y_train)
printModStats(model_xgb,X_train, Y_train,X_test, Y_test,showPlot=True)

In [ ]:
# Get only those observations which are protected areas
df_test_PA_raw = X_test_raw.loc[X_test_raw['wdpa_2017']==1,:]
print('Number of observations in test set that are protected:', df_test_PA_raw.shape[0])

In [ ]:
# Make a copy of those observations ...
df_test_PA_raw_ifNoPA = df_test_PA_raw.copy()
# ... and set the variable "protected areas" to zero
df_test_PA_raw_ifNoPA['wdpa_2017'] = 0

In [ ]:
# Use scaler to minmax scale data frames
df_test_PA = scaler.transform(df_test_PA_raw)
df_test_PA_ifNoPA = scaler.transform(df_test_PA_raw_ifNoPA)

In [ ]:
# Make a prediction with "wdpa_2017"=1
Y_hat_PA = model_xgb.predict(df_test_PA)
# Make a prediction with "wdpa_2017"=0
Y_hat_PA_idNoPA = model_xgb.predict(df_test_PA_ifNoPA)


In [ ]:

# Now we can compare the mean forest cover in protected area that we observer
print(f'Predicted mean forest cover in protected Areas {np.mean(Y_hat_PA):0.2f} %' )
print(f'Predicted mean forest cover in protected Areas assuming they are not protected {np.mean(Y_hat_PA_idNoPA):0.2f} %', )
print(f'Mean difference between the two: {np.mean(Y_hat_PA-Y_hat_PA_idNoPA):0.2f} pp')


In [ ]:
# => Discuss pro and cons of the two approaches